<a href="https://colab.research.google.com/github/bmpark16/nwHacks2026/blob/main/Procrastination_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Import and Install Dependencies

In [ ]:
!pip -q uninstall -y ydf tensorflow-decision-forests opentelemetry-proto grpcio-status mediapipe
!pip install mediapipe==0.10.14
!pip -q install -U "tensorflow==2.19.0" "keras==3.5.0"
!pip -q install -U opencv-python-headless scikit-learn matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 20.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 119.6 MB/s eta 0:00:00


In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [ ]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [ ]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [ ]:
def draw_styled_landmarks(image, results):
    # Draw face landmarks (using FaceMesh constants)
    if results.face_landmarks:
        mp_drawing.draw_landmarks(
            image,
            results.face_landmarks,
            mp.solutions.face_mesh.FACEMESH_CONTOURS,
            mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
            mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
        )

    # Draw pose connections
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_holistic.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
        )

    # Draw left hand connections
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(
            image,
            results.left_hand_landmarks,
            mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
        )

    # Draw right hand connections
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(
            image,
            results.right_hand_landmarks,
            mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
        )


In [ ]:
from google.colab import files
from google.colab.patches import cv2_imshow  # not used, but ok to keep
from IPython.display import HTML
import cv2
import os

# 1) Upload input video
uploaded = files.upload()
video_path = next(iter(uploaded))  # uploaded filename

# 2) Open input video
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise RuntimeError("Could not open uploaded video.")

# Read video properties
fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Some videos may report fps as 0 in certain environments; set a fallback
if fps is None or fps == 0:
    fps = 30.0

# 3) Create output writer
out_path = "annotated_output.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
writer = cv2.VideoWriter(out_path, fourcc, fps, (w, h))

# 4) Process frames with MediaPipe and write annotated frames
with mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret or frame is None:
            break

        # Make detections
        image, results = mediapipe_detection(frame, holistic)

        # Draw landmarks onto image
        draw_styled_landmarks(image, results)

        # Ensure frame is BGR for VideoWriter (your helper likely returns BGR already)
        # If your mediapipe_detection returns RGB, uncomment the next line:
        # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        writer.write(image)

cap.release()
writer.release()

print("Saved:", out_path)


StopIteration: 

In [ ]:
from base64 import b64encode

mp4 = open("annotated_output.mp4", "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=720 controls>
  <source src="{data_url}" type="video/mp4">
</video>
""")
files.download("annotated_output.mp4")

FileNotFoundError: [Errno 2] No such file or directory: 'annotated_output.mp4'

In [ ]:
if results.left_hand_landmarks:
    print(len(results.left_hand_landmarks.landmark))
else:
    print("No left hand landmarks detected in the last processed frame.")

AttributeError: 'NoneType' object has no attribute 'landmark'

In [ ]:
results

In [ ]:
draw_landmarks(frame, results)

In [ ]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

# 3. Extract Keypoint Values

In [ ]:
len(results.left_hand_landmarks.landmark)

NameError: name 'results' is not defined

In [ ]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

NameError: name 'results' is not defined

In [ ]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

NameError: name 'results' is not defined

In [ ]:
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten()
    if results.face_landmarks
    else np.zeros(1404)


IndentationError: unexpected indent (ipython-input-2479699165.py, line 2)

In [ ]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [ ]:
result_test = extract_keypoints(results)

In [ ]:
result_test

In [ ]:
468*3+33*4+21*3+21*3

In [ ]:
np.save('0', result_test)

In [ ]:
np.load('0.npy')

# 4. Setup Folders for Collection

In [42]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data')

# Actions that we try to detect
actions = np.array(['sleeping', 'doomscrolling'])

# Thirty videos worth of data
no_sequences = 0

# Videos are going to be 30 frames in length
sequence_length = 30

In [ ]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

In [ ]:
VIDEOS_PATH = "videos"          # change if your folder name differs
sequence_length = 30            # your fixed sequence length
max_videos_per_class = None     # set to an int for quick testing, e.g. 10

# Make sure MP_Data/<action>/<sequence>/ exists
for action in actions:
    action_path = os.path.join(DATA_PATH, action)
    os.makedirs(action_path, exist_ok=True)
    print(action) #fleming

def list_video_files(folder):
    exts = (".mp4", ".mov", ".m4v", ".avi", ".mkv")
    files = [f for f in os.listdir(folder) if f.lower().endswith(exts)]
    files.sort()
    return files

def save_video_as_keypoints(video_path, action, sequence_idx, holistic, sequence_length):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"[WARN] Could not open: {video_path}")
        return False

    # Make sequence folder
    seq_dir = os.path.join(DATA_PATH, action, str(sequence_idx))
    os.makedirs(seq_dir, exist_ok=True)

    keypoints_list = []

    # --- Read up to sequence_length frames (TRUNCATE if longer) ---
    while len(keypoints_list) < sequence_length:
        ret, frame = cap.read()
        if not ret:
            break  # video ended (too short)
        image, results = mediapipe_detection(frame, holistic)
        keypoints_list.append(extract_keypoints(results))

    cap.release()

    if len(keypoints_list) == 0:
        print(f"[WARN] No frames read from: {video_path}")
        return False

    # --- PAD with zeros if too short ---
    feature_dim = keypoints_list[0].shape[0]
    while len(keypoints_list) < sequence_length:
        keypoints_list.append(np.zeros(feature_dim, dtype=np.float32))

    # --- Save as .npy files (same as original notebook) ---
    for frame_num, keypoints in enumerate(keypoints_list[:sequence_length]):
        np.save(os.path.join(seq_dir, str(frame_num)), keypoints)

    return True

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        action_video_dir = os.path.join(VIDEOS_PATH, action)
        if not os.path.isdir(action_video_dir):
            raise FileNotFoundError(f"Missing folder: {action_video_dir}")

        video_files = list_video_files(action_video_dir)
        if max_videos_per_class is not None:
            video_files = video_files[:max_videos_per_class]

        print(f"\nProcessing {len(video_files)} videos for class '{action}'...")

        for sequence_idx, vf in enumerate(video_files):
            video_path = os.path.join(action_video_dir, vf)
            ok = save_video_as_keypoints(video_path, action, sequence_idx, holistic, sequence_length)
            if ok:
                print(f"  ✓ {vf} -> MP_Data/{action}/{sequence_idx}/[0..{sequence_length-1}].npy")
            else:
                print(f"  ✗ {vf} skipped")



error: OpenCV(4.12.0) /io/opencv/modules/imgproc/src/color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [ ]:
cap.release()
cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
label_map

In [ ]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
np.array(sequences).shape

In [ ]:
np.array(labels).shape

In [ ]:
X = np.array(sequences)

In [ ]:
X.shape

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [ ]:
y_test.shape

# 7. Build and Train LSTM Neural Network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
res = [.7, 0.2, 0.1]

In [ ]:
actions[np.argmax(res)]

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

In [ ]:
model.summary()

# 8. Make Predictions

In [ ]:
res = model.predict(X_test)

In [ ]:
actions[np.argmax(res[4])]

In [ ]:
actions[np.argmax(y_test[4])]

# 9. Save Weights

In [ ]:
model.save('action.h5')

In [ ]:
del model

In [ ]:
model.load_weights('action.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

# 11. Test in Real Time

In [ ]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

    return output_frame

In [ ]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

In [ ]:
sequence.reverse()

In [ ]:
len(sequence)

In [ ]:
sequence.append('def')

In [ ]:
sequence.reverse()

In [ ]:
sequence[-30:]

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)

        # Draw landmarks
        draw_styled_landmarks(image, results)

        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])


        #3. Viz logic
            if res[np.argmax(res)] > threshold:
                if len(sentence) > 0:
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5:
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)

        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
res[np.argmax(res)] > threshold

In [ ]:
(num_sequences,30,1662)

In [ ]:
model.predict(np.expand_dims(X_test[0], axis=0))

# Task
The current Colab notebook attempts to use OpenCV's `cv2.VideoCapture(0)` and `cv2.imshow` for webcam access and display, which are not directly supported in Colab. To enable webcam functionality, modify the existing data collection loop in cell `TBkWEy30nti2` to:
1.  Incorporate Colab-specific modules (`IPython.display`, `Javascript`, `google.colab.output`) and JavaScript code for capturing webcam frames.
2.  Implement a Python function to process base64 encoded frames received from the JavaScript into OpenCV images.
3.  Replace `cv2.VideoCapture(0)` and `cap.read()` with calls to this new webcam frame capture function.
4.  Replace `cv2.imshow` with `cv2_imshow` from `google.colab.patches` for displaying frames.

After these modifications, execute the updated data collection loop to successfully capture keypoint data from the webcam within Google Colab, ensuring proper handling for starting and stopping the feed.

## Prepare Colab for Webcam Access

### Subtask:
Import necessary Colab-specific modules (`IPython.display`, `Javascript`, `google.colab.output`) and define JavaScript code to capture webcam frames from the user's browser.


**Reasoning**:
The subtask requires importing specific modules for webcam interaction in Google Colab. This step imports `display`, `Javascript` from `IPython.display`, `output` from `google.colab`, and `cv2_imshow` from `google.colab.patches` to ensure all necessary Colab-specific functionalities for webcam interaction are available.



In [ ]:
from IPython.display import display, Javascript
from google.colab.output import output
from google.colab.patches import cv2_imshow

ImportError: cannot import name 'output' from 'google.colab.output' (/usr/local/lib/python3.12/dist-packages/google/colab/output/__init__.py)

**Reasoning**:
The `ImportError` indicates that `output` cannot be directly imported as a name from `google.colab.output`. Instead, `google.colab.output` is the module itself, and its functions like `send` are accessed as attributes of the module. Therefore, the correct way to import it is to import the module directly.



In [ ]:
from IPython.display import display, Javascript
import google.colab.output
from google.colab.patches import cv2_imshow

**Reasoning**:
Now that the necessary Python modules are imported, the next step is to define and embed the JavaScript functions for camera control (`init_camera` and `stop_camera`) using `IPython.display.Javascript`.



In [ ]:
def video_stream():
    js = Javascript('''
        var video;
        var div = null;
        var stream;
        var captureCanvas;
        var imgElement;
        var setup = (function() {
            var create ='<div style="border: 2px solid black; width: 100%; text-align: center; position: relative; display: inline-block;">';
            create += '<canvas id="captureCanvas" width=640 height=480 style="display: none;"></canvas>';
            create += '<video id="video" width=640 height=480 autoplay style="display: block;"></video>';
            create += '<button id="startStop" style="position: absolute; bottom: 10px; right: 10px;">Stop Stream</button>';
            create += '</div>';

            div = document.createElement('div');
            div.innerHTML = create;
            document.body.appendChild(div);

            video = document.getElementById('video');
            captureCanvas = document.getElementById('captureCanvas');
            imgElement = document.getElementById('imgElement');

            var startStopButton = document.getElementById('startStop');
            startStopButton.onclick = function() {
                if (stream) {
                    stop_camera();
                    startStopButton.innerText = 'Start Stream';
                } else {
                    init_camera();
                    startStopButton.innerText = 'Stop Stream';
                }
            };

            init_camera();

            async function init_camera() {
                stream = await navigator.mediaDevices.getUserMedia({ video: true });
                video.srcObject = stream;
                await video.play();

                google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
            }

            async function stop_camera() {
                if (!stream) return;
                stream.getTracks().forEach(track => { track.stop(); });
                video.srcObject = null;
                stream = null;
            }

            // Start capturing frames
            var send_frames = function() {
                if (stream) {
                    var ctx = captureCanvas.getContext('2d');
                    ctx.drawImage(video, 0, 0, captureCanvas.width, captureCanvas.height);
                    var data = captureCanvas.toDataURL('image/jpeg', 0.8);
                    google.colab.output.send({
                        'image': data
                    });
                }
            };

            setInterval(send_frames, 100); // Capture frame every 100ms
        })();
    ''')
    display(js)


def video_frame(b64_bytes):
    # Decode base64 image data
    import base64
    from io import BytesIO
    import cv2
    import numpy as np

    try:
        # The dataURL comes with a prefix 'data:image/jpeg;base64,', remove it
        encoded_data = b64_bytes.split(',')[1]
        nparr = np.frombuffer(base64.b64decode(encoded_data), np.uint8)
        img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
        return img
    except Exception as e:
        print(f"Error decoding frame: {e}")
        return None

video_stream()

<IPython.core.display.Javascript object>

**Reasoning**:
The previous code block successfully initialized the webcam stream and defined the `video_frame` function to process incoming frames. Now, the main data collection loop needs to be modified to use this new webcam functionality. This involves replacing `cv2.VideoCapture(0)` with a call to `google.colab.output.eval_js` to get frames, and `cv2.imshow` with `cv2_imshow` from `google.colab.patches`.



In [ ]:
import google.colab.output
import cv2

# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Get frame from webcam
                js_response = google.colab.output.eval_js('document.getElementById("captureCanvas").toDataURL("image/jpeg")')
                frame = video_frame(js_response)

                if frame is None:
                    print("Failed to get frame, skipping...")
                    continue

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)

                # NEW Apply wait logic
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120,200),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen using cv2_imshow
                    cv2_imshow(image)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen using cv2_imshow
                    cv2_imshow(image)

                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully - no cv2.waitKey needed for Colab's cv2_imshow loop exit
                # The JavaScript button 'Stop Stream' will handle stopping the feed
                # To manually stop the Python loop, you might need a different mechanism
                # For now, let it run through the defined number of sequences/frames


# Ensure all cv2_imshow windows are closed after the loop if any were created outside of this block
# In Colab, cv2_imshow doesn't create traditional windows that need closing like cv2.destroyAllWindows()
# The JavaScript stream will remain active until manually stopped or the cell execution ends.
print("Data collection complete. Stop the webcam stream using the 'Stop Stream' button or by executing the next cell.")

MessageError: TypeError: Cannot read properties of null (reading 'toDataURL')

**Reasoning**:
The `MessageError` indicates that `google.colab.output.eval_js` failed to access the JavaScript `captureCanvas` element, likely due to timing or execution context. The `video_stream` JavaScript is already set up to *push* frames using `google.colab.output.send`. To correctly receive these frames in Python, the data collection loop should use `google.colab.output.recv()` instead of `eval_js`. Additionally, `cv2.waitKey` is not suitable for `cv2_imshow` in Colab, so it will be replaced with `time.sleep` for introducing delays, and the loop variable `sequence` will be renamed to `sequence_idx` to avoid conflicts.



In [ ]:
import google.colab.output
import cv2
import time # Import time for sleep functionality

# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence_idx in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Get frame from webcam by receiving from JS
                # This will block until a message is received from JS
                message = google.colab.output.recv()
                if 'image' not in message:
                    print("Received non-image message, skipping...")
                    continue
                js_response = message['image']
                frame = video_frame(js_response)

                if frame is None:
                    print("Failed to get frame, skipping...")
                    continue

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)

                # NEW Apply wait logic
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120,200),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence_idx), (15,12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen using cv2_imshow
                    cv2_imshow(image)
                    time.sleep(2) # Use time.sleep for delay
                else:
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence_idx), (15,12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen using cv2_imshow
                    cv2_imshow(image)

                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence_idx), str(frame_num))
                np.save(npy_path, keypoints)

                # In Colab, cv2_imshow does not need cv2.waitKey for display management.
                # The JavaScript stream's setInterval controls the frame rate.

print("Data collection complete. Stop the webcam stream using the 'Stop Stream' button or by executing the next cell.")

AttributeError: module 'google.colab.output' has no attribute 'recv'

**Reasoning**:
The previous `video_stream` implementation using `google.colab.output.send` followed by `google.colab.output.recv` in Python failed because `recv` does not exist. This update modifies the JavaScript to continuously capture frames and store the latest one in a global variable (`window.latestFrame`). It also provides a global `window.stopColabWebcamStream` function for Python to stop the stream, making the frame retrieval synchronous via `eval_js('window.latestFrame')` possible.



In [ ]:
def video_stream():
    js = Javascript('''
        var video;
        var div = null;
        var stream;
        var captureCanvas;
        var latestFrame = null; // Global variable for the latest frame data
        var intervalId = null; // To store setInterval ID for clearing

        var setup = (function() {
            var create ='<div style="border: 2px solid black; width: 100%; text-align: center; position: relative; display: inline-block;">';
            create += '<canvas id="captureCanvas" width=640 height=480 style="display: none;"></canvas>';
            create += '<video id="video" width=640 height=480 autoplay style="display: block;"></video>';
            create += '<button id="startStop" style="position: absolute; bottom: 10px; right: 10px;">Stop Stream</button>';
            create += '</div>';

            div = document.createElement('div');
            div.innerHTML = create;
            document.body.appendChild(div);

            video = document.getElementById('video');
            captureCanvas = document.getElementById('captureCanvas');

            var startStopButton = document.getElementById('startStop');
            startStopButton.onclick = function() {
                if (stream) {
                    stop_stream(); // Changed from stop_camera to stop_stream
                    startStopButton.innerText = 'Start Stream';
                } else {
                    start_stream(); // Changed from init_camera to start_stream
                    startStopButton.innerText = 'Stop Stream';
                }
            };

            start_stream(); // Initial start

            async function start_stream() { // Renamed init_camera to start_stream for clarity
                stream = await navigator.mediaDevices.getUserMedia({ video: true });
                video.srcObject = stream;
                await video.play();

                intervalId = setInterval(send_frames, 100); // Start sending frames
                google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
            }

            async function stop_stream() { // Renamed stop_camera to stop_stream
                if (intervalId) {
                    clearInterval(intervalId); // Stop sending frames
                    intervalId = null;
                }
                if (stream) {
                    stream.getTracks().forEach(track => { track.stop(); });
                    video.srcObject = null;
                    stream = null;
                }
                latestFrame = null; // Clear the latest frame data
            }

            // Make stop_stream globally accessible for Python to call
            window.stopColabWebcamStream = stop_stream;

            // Start capturing frames and updating latestFrame
            var send_frames = function() {
                if (stream) {
                    var ctx = captureCanvas.getContext('2d');
                    ctx.drawImage(video, 0, 0, captureCanvas.width, captureCanvas.height);
                    latestFrame = captureCanvas.toDataURL('image/jpeg', 0.8); // Store data
                }
            };
        })();
    ''')
    display(js)


def video_frame(b64_bytes):
    # Decode base64 image data
    import base64
    from io import BytesIO
    import cv2
    import numpy as np

    try:
        # The dataURL comes with a prefix 'data:image/jpeg;base64,', remove it
        encoded_data = b64_bytes.split(',')[1]
        nparr = np.frombuffer(base64.b64decode(encoded_data), np.uint8)
        img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
        return img
    except Exception as e:
        print(f"Error decoding frame: {e}")
        return None

video_stream()

<IPython.core.display.Javascript object>

**Reasoning**:
The previous attempts to fetch frames using `google.colab.output.eval_js` directly from the canvas element and `google.colab.output.recv()` have failed. The current `video_stream` JavaScript has been updated to store the latest frame in `window.latestFrame`. Therefore, the data collection loop needs to be modified to retrieve frames from this global JavaScript variable using `google.colab.output.eval_js('window.latestFrame')`. Additionally, `cv2.waitKey` will be replaced with `time.sleep` for delays in Colab, and the loop variable `sequence` will be renamed to `sequence_idx` for clarity.



In [ ]:
import google.colab.output
import cv2
import time # Import time for sleep functionality

# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence_idx in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Get frame from webcam by evaluating JavaScript to get the latest frame
                js_response = google.colab.output.eval_js('window.latestFrame')

                if js_response == 'null': # Check if frame data is available yet
                    print("Waiting for first frame...")
                    time.sleep(0.1)
                    continue

                frame = video_frame(js_response)

                if frame is None:
                    print("Failed to decode frame, skipping...")
                    continue

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)

                # NEW Apply wait logic
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120,200),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence_idx), (15,12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen using cv2_imshow
                    cv2_imshow(image)
                    time.sleep(2) # Use time.sleep for delay
                else:
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence_idx), (15,12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen using cv2_imshow
                    cv2_imshow(image)

                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence_idx), str(frame_num))
                np.save(npy_path, keypoints)

                # Add a small delay to control frame rate for display, if necessary
                time.sleep(0.03) # Roughly 30 fps

# Call the JavaScript function to stop the webcam stream cleanly
google.colab.output.eval_js('window.stopColabWebcamStream();')
print("Data collection complete. Webcam stream stopped.")

Error decoding frame: 'NoneType' object has no attribute 'split'
Failed to decode frame, skipping...
Error decoding frame: 'NoneType' object has no attribute 'split'
Failed to decode frame, skipping...
Error decoding frame: 'NoneType' object has no attribute 'split'
Failed to decode frame, skipping...
Error decoding frame: 'NoneType' object has no attribute 'split'
Failed to decode frame, skipping...
Error decoding frame: 'NoneType' object has no attribute 'split'
Failed to decode frame, skipping...
Error decoding frame: 'NoneType' object has no attribute 'split'
Failed to decode frame, skipping...
Error decoding frame: 'NoneType' object has no attribute 'split'
Failed to decode frame, skipping...
Error decoding frame: 'NoneType' object has no attribute 'split'
Failed to decode frame, skipping...
Error decoding frame: 'NoneType' object has no attribute 'split'
Failed to decode frame, skipping...
Error decoding frame: 'NoneType' object has no attribute 'split'
Failed to decode frame, sk

KeyboardInterrupt: 